In [38]:
import pandas as pd
import numpy as np
pd.set_option('chained_assignment', None)

#path = "C:\\Users\\dmehri\\Documents\\DATA\\FDNY\\"
path = "C:\\Users\\DOBUSER\\Documents\\DATA\\FDNY\\"

print ("Read line of duty deaths ...")
df = pd.read_csv(path + "FDNY_Line_Of_Duty_Deaths.csv")

print ("Number of line of duty deaths since 1865:", len(df))

print ("Remove outliers - 911 deaths...")
df = df[df["Date"] != "September 11, 2001"].reset_index(drop=True)

print ("Number of line of duty deaths after removing 911:", len(df))



df['Date'] = pd.to_datetime(df['Date'])

# just get year
df['Year'] = pd.DatetimeIndex(df['Date']).year

print ("Creating decade...")

df["Decade"] = ""

for i in range(0, len(df)):
    year = df['Year'][i]
    if (year >= 1860 and year <= 1869):
        df["Decade"][i] = "1860s"
    if (year >= 1870 and year <= 1879):
        df["Decade"][i] = "1870s"
    if (year >= 1880 and year <= 1889):
        df["Decade"][i] = "1880s"
    if (year >= 1890 and year <= 1899):
        df["Decade"][i] = "1890s"
    if (year >= 1900 and year <= 1909):
        df["Decade"][i] = "1900s"
    if (year >= 1910 and year <= 1919):
        df["Decade"][i] = "1910s"
    if (year >= 1920 and year <= 1929):
        df["Decade"][i] = "1920s"
    if (year >= 1930 and year <= 1939):
        df["Decade"][i] = "1930s"
    if (year >= 1940 and year <= 1949):
        df["Decade"][i] = "1940s"
    if (year >= 1950 and year <= 1959):
        df["Decade"][i] = "1950s"
    if (year >= 1960 and year <= 1969):
        df["Decade"][i] = "1960s"
    if (year >= 1970 and year <= 1979):
        df["Decade"][i] = "1970s"
    if (year >= 1980 and year <= 1989):
        df["Decade"][i] = "1980s"
    if (year >= 1990 and year <= 1999):
        df["Decade"][i] = "1990s"
    if (year >= 2000 and year <= 2009):
        df["Decade"][i] = "2000s"
    if (year < 1860 or year > 2009):
        df["Decade"][i] = "nan"
print ("DONE")

Read line of duty deaths ...
Number of line of duty deaths since 1865: 1169
Remove outliers - 911 deaths...
Number of line of duty deaths after removing 911: 826
Creating decade...
DONE


### INSERT COMMUNITY BOARD AND LATITUDE AND LONGITUDE COORDINATES INTO LINE OF DUTY DEATHS

In [39]:
print ("Read fire house listings ...")
dfh = pd.read_csv(path + "FDNY_Firehouse_Listing.csv")


df["Unit"] = df["Unit"].astype(str).map(str.strip).map(str.upper)
dfh["FacilityName"] = dfh["FacilityName"].astype(str).map(str.strip).map(str.upper)


#correct for format in unit 1/unit 2/unit 3
dfh["Unit1"] = dfh['FacilityName'].str.split("/").str[0]
dfh["Unit2"] = dfh['FacilityName'].str.split("/").str[1]
dfh["Unit3"] = dfh['FacilityName'].str.split("/").str[2]

unit1 = dfh[["Unit1", "Latitude", "Longitude", "Community Board"]]
unit1 = unit1.dropna(subset = ['Unit1'])
print (len(dfh), len(unit1))

unit2 = dfh[["Unit2", "Latitude", "Longitude", "Community Board"]]
unit2 = unit2.dropna(subset = ['Unit2'])
print (len(dfh), len(unit2))

unit3 = dfh[["Unit3", "Latitude", "Longitude", "Community Board"]]
unit3 = unit3.dropna(subset = ['Unit3'])
print (len(dfh), len(unit3))

unit1["Unit1"] = unit1["Unit1"].astype(str).map(str.strip).map(str.upper)
unit2["Unit2"] = unit2["Unit2"].astype(str).map(str.strip).map(str.upper)
unit3["Unit3"] = unit3["Unit3"].astype(str).map(str.strip).map(str.upper)

unit1lat = unit1.set_index('Unit1')['Latitude'].to_dict()
unit1lon = unit1.set_index('Unit1')['Longitude'].to_dict()
unit1cb = unit1.set_index('Unit1')['Community Board'].to_dict()


unit2lat = unit2.set_index('Unit2')['Latitude'].to_dict()
unit2lon = unit2.set_index('Unit2')['Longitude'].to_dict()
unit2cb = unit2.set_index('Unit2')['Community Board'].to_dict()


unit3lat = unit3.set_index('Unit3')['Latitude'].to_dict()
unit3lon = unit3.set_index('Unit3')['Longitude'].to_dict()
unit3cb = unit3.set_index('Unit3')['Community Board'].to_dict()


print ("Groupby unit to get count of deaths per unit...")
df2  = df[["Unit", "Decade"]]
df2["Deaths"] = 1
df2 = df2.groupby(['Unit', 'Decade']).sum()
df2 = df2.add_suffix('').reset_index()

df2["Latitude"] = df2["Unit"].map(unit1lat)
df2["Longitude"] = df2["Unit"].map(unit1lon)
df2["Community Board"] = df2["Unit"].map(unit1cb)


#df2["Latitude"] = df2["Unit"].map(unit2lat)
#df2["Longitude"] = df2["Unit"].map(unit2lon)

df2["Latitude"] = df2["Latitude"].astype(str)
df2["Longitude"] = df2["Longitude"].astype(str)

count2=0
count3=0
for i in range(0, len(df2)):
    theunit = df2["Unit"][i]
    if (df2["Latitude"][i] == 'nan' and theunit in unit2lat):
        #print (theunit)
        df2["Latitude"][i] = unit2lat[theunit]
        df2["Longitude"][i] = unit2lon[theunit]
        df2["Community Board"][i] = unit2cb[theunit]

        count2+=1
    if (df2["Latitude"][i] == 'nan' and theunit in unit3lat):   
        df2["Latitude"][i] = unit3lat[theunit]
        df2["Longitude"][i] = unit3lon[theunit]
        df2["Community Board"][i] = unit3cb[theunit]
        count3+=1
        
print (count2)
print (count3)




Read fire house listings ...
219 219
219 156
219 42
Groupby unit to get count of deaths per unit...
209
70


In [41]:
df2.head(3)

,Unit,Decade,Deaths,Latitude,Longitude,Community Board
0,AMBULANCE 1,1970s,1,nan,nan,NaN
1,BATTALION 11,1950s,1,40.79650141,-73.96719303,107.0
2,BATTALION 11,1970s,1,40.79650141,-73.96719303,107.0


### GROUP BY CB, LAT LON AND COUNT DEATHS

In [76]:
#df3  = df2[["Community Board", "Deaths"]]
#df3 = df3.groupby(['Community Board']).sum()
#df3 = df3.add_suffix('').reset_index()
#output = df2[df2["Latitude"] != "nan"]
#df3.to_csv(path + "FDNY Line of Duty Deaths by Community Board.csv", index=False)

In [12]:
df2.head(3)

,Unit,Decade,Deaths,Latitude,Longitude,Community Board
0,AMBULANCE 1,1970s,1,nan,nan,NaN
1,BATTALION 11,1950s,1,40.79650141,-73.96719303,107.0
2,BATTALION 11,1970s,1,40.79650141,-73.96719303,107.0


In [67]:
df2["Latitude"] = df2["Latitude"].astype(str)
df2["Longitude"] = df2["Longitude"].astype(str)
df2["lat_lon"] = df2["Latitude"] + df2["Longitude"]

df3 = df2[["lat_lon","Decade", "Deaths"]]
df3 = df3.groupby(['lat_lon', 'Decade']).sum()
df3 = df3.add_suffix('').reset_index()

#ADD LAT LON TO GROUPBY
lat = df2[["lat_lon", "Latitude"]]
lat = lat.set_index('lat_lon')['Latitude'].to_dict()
df3["Latitude"] = df3["lat_lon"].map(lat)
lon = df2[["lat_lon", "Longitude"]]
lon = lon.set_index('lat_lon')['Longitude'].to_dict()
df3["Longitude"] = df3["lat_lon"].map(lon)

#ADD UNITS
units = df2[["lat_lon", "Unit"]]
units = units.groupby('lat_lon')['Unit'].apply(lambda x: "{%s}" % ', '.join(x))


units = pd.DataFrame(units)
units = units.reset_index(drop=False)
units['Unit'] = units['Unit'].str.replace('{', '')
units['Unit'] = units['Unit'].str.replace('}', '')
units = units.set_index('lat_lon')['Unit'].to_dict()
df3["Units"] = df3["lat_lon"].map(units)

#remove duplicate units
for i in range(0, len(df3)):
    a = df3["Units"][i]
    a = a.split(", ")
    a = sorted(list(set(a)) )
    #print (a)
    df3["Units"][i] = a
df3["Units"] = df3["Units"].astype(str)
df3["Units"] = df3["Units"].str.replace('[', '')
df3["Units"] = df3["Units"].str.replace(']', '')
df3["Units"] = df3["Units"].str.replace("'", '')

print (len(df3))
df3["Decade"] = df3["Decade"].astype(str)
df3 = df3[df3["Decade"] != "nan"]
print (len(df3))
df3["lat_lon"] = df3["lat_lon"].astype(str)
df3 = df3[df3["lat_lon"] != "nannan"]
print (len(df3))

538
537
524


In [68]:
df3.to_csv(path + "FDNY Line of Duty Deaths Geodata w Decade.csv", index=False)